In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import google.generativeai as genai
import joblib  # For saving and loading the entire object

# ================================
# 🚀 Define Crop Recommendation Class
# ================================
class CropRecommender:
    def __init__(self):
        """Initialize model and scaler."""
        self.model = RandomForestClassifier(n_estimators=20, random_state=0)
        self.scaler = StandardScaler()

    def train(self, file_path):
        """Train the model using dataset from file_path."""
        df = pd.read_csv(file_path)
        X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
        y = df['label']

        # Split dataset
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Standardize features
        self.scaler.fit(X_train)
        X_train_scaled = self.scaler.transform(X_train)

        # Train model
        self.model.fit(X_train_scaled, y_train)

        print("✅ Model trained and stored inside the class.")

    def predict_crop(self, N, P, K, temperature, humidity, ph, rainfall):
        """Predict the crop based on soil and climate input."""
        input_data = np.array([[N, P, K, temperature, humidity, ph, rainfall]])
        input_data_scaled = self.scaler.transform(input_data)
        predicted_crop = self.model.predict(input_data_scaled)[0]
        return predicted_crop

    def ask_gemini(self, question):
        """Query Gemini AI for additional insights."""
        genai.configure(api_key="Enter your API key here")  #
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(question)
        return response.text if response else "I'm unable to fetch an answer right now."

    def interactive_prediction(self):
        """Interactive CLI for user input and recommendations."""
        print("\n💬 Enter soil and climate details to get crop recommendations\n")

        try:
            N = float(input("🌱 Nitrogen (N) content: "))
            P = float(input("🌾 Phosphorus (P) content: "))
            K = float(input("🍀 Potassium (K) content: "))
            temperature = float(input("🌡️ Temperature (°C): "))
            humidity = float(input("💧 Humidity (%): "))
            ph = float(input("🧪 Soil pH Level: "))
            rainfall = float(input("🌧️ Rainfall (mm): "))

            # Get recommendation
            predicted_crop = self.predict_crop(N, P, K, temperature, humidity, ph, rainfall)
            print(f"\n🌾 **Recommended Crop:** {predicted_crop} ✅\n")

            # Ask if the user wants a different crop
            choice = input("\n🤔 Do you want to grow a different crop? (yes/no): ").strip().lower()
            if choice == "yes":
                other_crop = input("🌱 Enter the crop you wish to grow: ").strip()
                query = f"What changes should a farmer make to grow {other_crop} instead of {predicted_crop}?"
                print("\n💡 Advice from Gemini AI:")
                print(self.ask_gemini(query))

            # Ask about challenges for the recommended crop
            challenge_query = f"What are the biggest challenges in growing {predicted_crop}?"
            print("\n⚠️ Challenges of growing", predicted_crop, ":\n", self.ask_gemini(challenge_query))

            # Ask about ideal conditions for the crop
            ideal_conditions_query = f"What are the ideal growing conditions for {predicted_crop}?"
            print("\n🌱 Ideal conditions for", predicted_crop, ":\n", self.ask_gemini(ideal_conditions_query))

        except ValueError:
            print("⚠️ Invalid input! Please enter numerical values only.")

# ================================
# 🚀 Train and Save the Class as a `.pkl` File
# ================================
recommender = CropRecommender()
recommender.train("/content/crop_recommendation.csv")  # Train the model

# Save the entire class object in a `.pkl` file
joblib.dump(recommender, "crop_recommendation.pkl")
print("✅ Model and all functions saved inside 'crop_recommendation.pkl'.")



✅ Model trained and stored inside the class.
✅ Model and all functions saved inside 'crop_recommendation.pkl'.


In [3]:
import joblib

# Load the trained CropRecommender object
recommender = joblib.load("crop_recommendation.pkl")

# Run interactive prediction
recommender.interactive_prediction()



💬 Enter soil and climate details to get crop recommendations

🌱 Nitrogen (N) content: 90
🌾 Phosphorus (P) content: 42
🍀 Potassium (K) content: 43
🌡️ Temperature (°C): 27
💧 Humidity (%): 81
🧪 Soil pH Level: 6
🌧️ Rainfall (mm): 200


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



🌾 **Recommended Crop:** rice ✅


🤔 Do you want to grow a different crop? (yes/no): yes
🌱 Enter the crop you wish to grow: maze

💡 Advice from Gemini AI:
**Tillage and Soil Preparation:**

* **Reduce waterlogging:** Maize requires well-drained soils. Rice paddies, on the other hand, are flooded. Drainage systems or raised beds may be needed to prevent waterlogging.
* **Deep plowing:** Maize has deep roots, so deeper plowing is required to allow for root development.

**Planting and Seed Selection:**

* **Plant spacing:** Maize is typically planted in rows with wider spacing than rice. This allows for air circulation and prevents overcrowding.
* **Seed selection:** Choose maize varieties suitable for the climate and soil conditions in your area. Opt for hybrids that offer resistance to pests and diseases.

**Fertilization:**

* **Nitrogen:** Maize requires higher levels of nitrogen fertilization than rice. Soil tests can help determine specific application rates.
* **Phosphorus and pota